In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [124]:
# 原始所有歌手的清單
df_artists = pd.read_excel(r'csv/20200303/artist_list_final.xlsx')
df_artists

,Artist
0,5 Seconds of Summer
1,Aaron Carter
2,Aaron Lohr
3,Abraham Mateo
4,Abraham Quintanilla Jr.
...,...
569,Zabdiel De Jesus
570,Zach Herron
571,Zara Larsson
572,Zendaya Maree Stoermer Coleman


In [130]:
# spotifyAPI存取權限
# access token要隨時更新 (https://spotify-token-finder.herokuapp.com/)
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
access_token = 'BQAzq-LmH4xOVvIC3wdPEMJKQ48_a0vihS_1VucyUjT1dK3iiSl4V507l2oyUD9i1awT__v5A8He3krfsLU'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [126]:
# 取得歌手資訊
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    # 歌手有items資訊
    if items:
        data = items[0]
        if len(items) > 0:
            artist_name = items[0]['name']
            artist_id = items[0]['id']
            artist_url = items[0]['external_urls']['spotify']
            artist_info = (artist_name,artist_id,artist_url)
            #print(artist_info)
            return items[0]
        else:
            return None
    # 歌手沒有items資訊，則回傳check值為1
    else:
        print('None items')
        check = 1
        return check
        pass

In [127]:
# 取得歌手名稱和類型
# https://github.com/Leats/spotify-historymaker/blob/master/spotifyhistorymaker.py
# Bruno Mars = 0du5cEVh5yTK9QJze8zA0C / Adele = 4dpARuHxo51G3z768sgnrY
def get_artist_name_genres(access_token: str, artist_id: str):    
    bearer_token = f'Bearer {access_token}'
    track_headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': bearer_token}
    response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=track_headers)
    response.raise_for_status()
    getname = response.json()['name'] # 型態str
    getid = response.json()['id'] # 型態str
    getgenres = response.json()['genres'] # 型態list
    getgenres2 = ','.join(getgenres) # 型態str
    
    list_artist_name = []
    list_artist_id = []
    list_artist_genres = []
    list_artist_name.append(getname)
    list_artist_id.append(getid)
    list_artist_genres.append(getgenres2)
    return list_artist_genres

In [141]:
# 輸出歌手名稱+Id+Genres的csv(artist_list_ID+Genres_版本.csv)
with open("csv/20200303/v1/artist_list_ID+Genres_v1.csv", "a", encoding="utf_8_sig", newline='') as csvfile:
    writer = csv.writer(csvfile)
    #第一次寫入:w，之後寫入:a
    #writer.writerow(['Artist','Id','Genres'])
    df_artists = pd.read_excel(r'csv/20200303/artist_list_final.xlsx')
    base = 500
    df = df_artists[['Artist']][500:] #base:base+100
    L_artist_name = []
    L_artist_id = []
    L_artist_genres = []
    no_item = []
    for i in range(len(df)):
        artist_name = df['Artist'][base+i]
        print(str(i) + '|' + str(artist_name))
        # 判斷check值是否為1(代表歌手不存在)
        check = get_artist(artist_name)
        if check != 1:         
            try:
                artist_info = get_artist(artist_name)
                artist_id = artist_info['id']
                #print(artist_id)
                list_artist_genres = get_artist_name_genres(access_token,artist_id)
                artist_genres = ",".join(list_artist_genres) # list >>>str
                L_artist_name.append(artist_name)
                L_artist_id.append(artist_id)
                L_artist_genres.append(list_artist_genres)
            except:
                L_artist_id.append('')
                L_artist_genres.append('')
                pass
            #之後寫入
            writer.writerow([artist_name, artist_id, artist_genres])
        else:
            no_item.append(artist_name)
            pass

0|Selena Gomez
1|Sergio Calderon
2|Shaggy
3|Shaheen Jafargholi
None items
4|Shakira
5|Sham Idrees
6|Shane Harper
7|Shania Twain
8|Shaun Cassidy
9|Shawn Mendes
10|Sheb Wooley
11|Sheena Easton
12|Sia Furler
13|Sianney Garcia
None items
14|Sierra Deaton
None items
15|Sigrid Raabe
16|Sinead O'Connor
17|Skylar Grey
18|Smokey Robinson
19|Sofia Daccarett Char
None items
20|Solange Knowles
21|Sonny Bono
22|Sophia Grace Brownlee
23|Steve Lawrence
24|Steven Tyler
25|Stevie Wonder
26|Sunny Malouf
27|Susan Boyle
28|Sydney Sierota
29|Tammy Townsend
30|Tank
31|Tayla Parx
32|Taylor Swift
33|Teairra Mari
34|Telma Hopkins
None items
35|Teyana Taylor
36|That Poppy
37|The Chainsmokers
retrying ...1secs
38|The Weeknd
39|Thomas Mars
40|Tiffany Alvord
41|Tina Turner
42|Tionne Watkins
None items
43|Tito Jackson
44|Toby McDonough
45|Todd Rundgren
46|Toni Braxton
47|Tony Bennett
48|Tony Oller
49|Tori Amos
50|Tori Kelly
51|Tracy Chapman
52|Trevor Jackson
53|Trey Schafer
54|Tricia Leigh Fisher
55|Trinitee Stokes

In [143]:
# 要保留的歌手類型清單
df_genres_select = pd.read_excel(r'csv/20200303/artist_genres_select.xlsx')
df_genres_select.drop('Unnamed: 0',axis=1,inplace=True)
df_genres_select

,Genres
0,pop
1,dance pop
2,post-teen pop
3,urban contemporary
4,indie poptimism
...,...
1725,swedish choir
1726,cathedral choir
1727,atmosphere
1728,wind quintet


In [144]:
# 讀取artist_list_ID+Genres_版本.csv
df_id_genres = pd.read_csv(r'csv/20200303/v1/artist_list_ID+Genres_v1.csv')
df_id_genres

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Aaron Lohr,7kpYFxC5CpMUjfXtOGOrvR,NaN
3,Abraham Mateo,2bxxlINUlcMQQb39K7IopR,"latin,latin arena pop,latin pop,latin viral po..."
4,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
...,...,...,...
489,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
490,Ylona Garcia,2qnQOnL1oLKtH779qZ6iuR,NaN
491,Zabdiel De Jesus,1eaadKSUwsMHA3w0P6g2sk,NaN
492,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [150]:
# 刪除Genres為空的那一列
df_id_genres_drop = df_id_genres.dropna(axis=0, subset=['Genres'])
df_id_genres_drop.reset_index(inplace=True,drop=True)
df_id_genres_drop.to_csv('csv/20200303/v1/artist_list_ID+Genres_drop_v1.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Abraham Mateo,2bxxlINUlcMQQb39K7IopR,"latin,latin arena pop,latin pop,latin viral po..."
3,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
4,Adam Levine,4bYPcJP5jwMhSivRcqie2n,pop rock
...,...,...,...
340,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
341,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop,post-tee..."
342,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
343,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [152]:
# 逐行查看歌手所屬類別是否在篩選後的總類別中(artist_genres_select.xlsx)
count = 0
delete_index = []
df_id_genres_drop = pd.read_csv(r'csv/20200303/v1/artist_list_ID+Genres_drop_v1.csv')
for row in range(len(df_id_genres_drop)):
    # 把歌手類型清單的類型欄位轉換成list
    # https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_datafra
    List1 = df_genres_select['Genres'].tolist() # 總類別清單
    List2 = df_id_genres_drop['Genres'][row].split(',') # 每一位歌手的類別清單
    
    # 檢查歌手的類別有沒有包含在總類別中
    check = any(item in List1 for item in List2)
    
    # 有包含
    if check is True:
        # 原始型態set，轉成list型態，印出有交集的類別
        # https://www.geeksforgeeks.org/python-convert-set-into-a-list/
        set1 = set(List1) 
        set2 = set(List2)
        print(str(count) + ' : ' + 'List1 ⊂ List2' + '\n' + 'Contain Genres:' + str(list(set1.intersection(set2))) + '\n' + '-'*90)
    # 沒包含
    else:
        print(str(count) + ' : ' + 'List1 ⊄ List2' + '\n' + '-'*90)
        delete_index.append(count)
        pass
    count += 1

print('Delete Index:',delete_index)

0 : List1 ⊂ List2
Contain Genres:['pop', 'post-teen pop', 'boy band', 'dance pop']
------------------------------------------------------------------------------------------
1 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop']
------------------------------------------------------------------------------------------
2 : List1 ⊂ List2
Contain Genres:['latin arena pop', 'latin', 'spanish pop', 'latin viral pop']
------------------------------------------------------------------------------------------
3 : List1 ⊂ List2
Contain Genres:['dance pop', 'pop', 'post-teen pop', 'idol']
------------------------------------------------------------------------------------------
4 : List1 ⊄ List2
------------------------------------------------------------------------------------------
5 : List1 ⊄ List2
------------------------------------------------------------------------------------------
6 : List1 ⊂ List2
Contain Genres:['pop', 'uk pop']
---------------------------------------------

In [154]:
# 篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
 # https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop.drop(df_id_genres_drop.index[[delete_count]],inplace=True)
df_id_genres_drop.reset_index(drop=True, inplace=True)
df_id_genres_drop.to_csv('csv/20200303/v1/artist_list_ID+Genres_select_v1.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Adam Rickitt,2u1BQaJtganAtbdBrUPZe3,bubblegum dance
3,Agnetha Faltskog,7fUtt9kVZOyn9LWy0JbDRI,"classic swedish pop,classic uk pop,europop,new..."
4,Agnez Mo,6pMR8Zgot664613rAiLC2Z,indonesian pop
...,...,...,...
322,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
323,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop,post-tee..."
324,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
325,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [56]:
# 把歌手類型清單的類型欄位轉換成list
# https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_dataframe
List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
#print('List1 =',List1)
print(len(List1))
#print('List2 =',List2)
print(len(List2))

if len(List1) == len(df_genres_select):
    print('Same')

check =  any(item in List1 for item in List2)
if check is True:
    print("List1 ⊂ List2")    
else :
    print("List1 ⊄ List2")

1730
4
Same
The list List1 contains some elements of the list List2


In [60]:
# list取交集的部分
def list_contains(List1, List2):  
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        print('Contain Genres:',list(set1.intersection(set2))) # 原始型態set
        return True 
    else: 
        return False

List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
list_contains(List1, List2)

Contain Genres: ['pop', 'post-teen pop', 'boy band', 'dance pop']


True